In [ ]:
import os
import re
import time
import json
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import random
from torch_geometric.data import Data
import pybullet as p
import pybullet_data
import trimesh
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import defaultdict, Counter

from core.env.scene_manager import plot_scene, cal_density

In [ ]:
from core.sim.utils import (
    Camera, PyBulletSim, load_object, load_table, get_size, random_tilt
)

sim = PyBulletSim(p.GUI)
# sim = PyBulletSim(p.DIRECT)
cam = Camera(target_pos=[0.5, 0, 1], distance=0.1, yaw=90, pitch=-40, roll=0)
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)
table = load_table((100, 100))
z = get_size(table)[2]
load_object('basket', [0.3, -0.15, z+0.05], bodyType=3, useFixedBase=True)
# mug_orn = [0, 0, random.uniform(0, 2 * np.pi)]
mug_orn = [0, 0, np.pi/1.5]
load_object('mug',    [0.3, -0.15, z+0.1], mug_orn, bodyType=3)
pos, orn = [0.3, -0.15, z+0.2], [np.pi / 2, 0, 0]
pos, orn = random_tilt(pos, orn, max_shift=0.01, tilt_angle=10)
load_object('spoon',  pos, orn)

sim.step(20)
load_object('mug',    [0.3, 0.15, z+0.08], mug_orn, bodyType=3)
load_object('spoon',  pos+[0, 0.3, 0], orn)
sim.close()

# Scene Load

In [ ]:
import pathlib
pathlib.PosixPath = pathlib.WindowsPath

model_path = f'perception/final_yolov5l/weights/best.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True, device='cpu')

In [ ]:
img_unseen = Image.open(f"dataset/rearrangement_00001/initial_image.png")
results = model(img_unseen)
# results.pandas().xyxy
results.show()

# Simulation

In [ ]:
import roboticstoolbox as rtb
from core.sim.utils import (
    Camera, PyBulletSim, load_object, load_table, get_size,
    Robot
)

def random_pos(x_range, y_range, z_range):
    return [
        random.uniform(*x_range),
        random.uniform(*y_range),
        random.uniform(*z_range)
    ]

def multiply_transforms(posA, ornA, posB, ornB):
    """Combine two transforms."""
    return p.multiplyTransforms(posA, ornA, posB, ornB)

def invert_transform(pos, orn):
    """Invert a transform."""
    return p.invertTransform(pos, orn)

def grasp_obj(robot, obj_id):
	robot.open_gripper()
	obj_pos, obj_orn = p.getBasePositionAndOrientation(obj_id)
	robot.go_to_pose([obj_pos[0], obj_pos[1], obj_pos[2]+0.3])
	robot.go_to_pose([obj_pos[0], obj_pos[1], obj_pos[2]+0.1])

	# Create the fixed constraint with relative pose
	ee_pos, ee_orn = robot.get_ee_pos()
	inv_ee_pos, inv_ee_orn = invert_transform(ee_pos, ee_orn)
	_, rel_orn = multiply_transforms(inv_ee_pos, inv_ee_orn, obj_pos, obj_orn)
	constraint_id = p.createConstraint(
		parentBodyUniqueId=robot_id,
		parentLinkIndex=11,
		childBodyUniqueId=obj_id,
		childLinkIndex=-1,
		jointType=p.JOINT_FIXED,
		jointAxis=[0, 0, 0],
		parentFramePosition=[0, 0, 0],
		childFramePosition=[0, 0, 0.1],
		childFrameOrientation=rel_orn
	)
	# robot.go_to_pose([obj_pos[0], obj_pos[1], obj_pos[2]])
	robot.close_gripper()
	
	robot.go_to_pose([obj_pos[0], obj_pos[1], obj_pos[2]+0.3])

	return constraint_id
	# return None

def release_obj(robot, pos, constraint_id=None):
	robot.go_to_pose([pos[0], pos[1], pos[2]+0.3])
	robot.go_to_pose([pos[0], pos[1], pos[2]+0.1])
	robot.open_gripper()
	if constraint_id is not None:
		p.removeConstraint(constraint_id)

sim = PyBulletSim(p.GUI)
# sim = PyBulletSim(p.DIRECT)
table = load_table((100, 100))
z = get_size(table)[2]
x_range, y_range, z_range = [-0.3, 0.3], [-0.3, 0.3], [z+0.05, z+0.05]
cam = Camera(target_pos=[0.6, 0, 1.3], distance=0.5, yaw=90, pitch=-30, roll=0)

robot = Robot(p, "franka_panda/panda.urdf", rtb.models.Panda(), [-0.6, 0, z])
robot_id = robot.init('stationary', x_range, y_range)
init_pos = robot.get_ee_pos()[0]

apple1 = load_object('pear', random_pos(x_range, y_range, z_range), bodyType=1)[0]
apple2 = load_object('pear', random_pos(x_range, y_range, z_range), bodyType=2)[0]
apple3 = load_object('pear', random_pos(x_range, y_range, z_range), bodyType=3)[0]
cam.show_img(cam.capture_image())

constraint_id = grasp_obj(robot, apple1)
cam.show_img(cam.capture_image())
release_obj(robot, random_pos(x_range, y_range, z_range), constraint_id)
cam.show_img(cam.capture_image())
robot.go_to_pose(init_pos)
robot.reset_joints()
cam.show_img(cam.capture_image())

constraint_id = grasp_obj(robot, apple2)
cam.show_img(cam.capture_image())
release_obj(robot, random_pos(x_range, y_range, z_range), constraint_id)
cam.show_img(cam.capture_image())
robot.go_to_pose(init_pos)
robot.reset_joints()
cam.show_img(cam.capture_image())

constraint_id = grasp_obj(robot, apple3)
cam.show_img(cam.capture_image())
release_obj(robot, random_pos(x_range, y_range, z_range), constraint_id)
cam.show_img(cam.capture_image())
robot.go_to_pose(init_pos)
robot.reset_joints()
cam.show_img(cam.capture_image())

sim.close()